In [4]:
import numpy as np
import tensorflow as tf
import json
from datetime import datetime
import os
import sys
sys.path.append('../')
import retriever

In [5]:
trainset_file = '../data/training/train_data.bin'
testset_file = '../data/training/test_data.bin'
vocab_file = '../data/metadata/w2v_vocab.json'
# params_dir = '../data/training/models/GroundR/params:hs,50:lr,0.05'
params_dir_tmp = '../data/training/models/RGAB5/All/'

In [6]:
trainset = np.load(open(trainset_file, 'rb'))
trainset = [item for item in trainset if len(item)>2 and len(item[0])>0]
print('trainset length:', len(trainset))

testset = np.load(open(testset_file, 'rb'))
testset = [item for item in testset if len(item)>2 and len(item[0])>0]
print('test length:', len(testset))

with open(vocab_file, 'r') as f:
    vocab = json.loads(f.read())

trainset length: 59449
test length: 59507


In [7]:
vocab['<unk>']

8915

In [8]:
def accuracy(data, batch_size, model):
    np.random.shuffle(data)
    np.random.shuffle(data)
    nbatch = len(data)//batch_size
    print('Number of batches:', nbatch, '\n')
    acc = 0
    stime, btime = datetime.now(), datetime.now()
    for b in range(nbatch):
        b_acc=model.accuracy(data, b*batch_size, (b+1)*batch_size)
        acc+=b_acc/nbatch
        if b%100==0:
            print('-batch number:', b, ' '*(len(str(nbatch))-len(str(b))), '-- accuracy:', b_acc, '-- time interval:', datetime.now()-btime, '-')
            btime = datetime.now()
    print('\nAccuracy:', acc, '\nTime i+nterval:', datetime.now()-stime)
    return acc

def iou_accuracy(data, batch_size, model, threshold=0.5):
    np.random.shuffle(data)
    np.random.shuffle(data)
    nbatch = len(data)//batch_size
    print('Number of batches:', nbatch, '\n')
    acc = 0
    stime, btime = datetime.now(), datetime.now()
    for b in range(nbatch):
        b_acc=model.iou_accuracy(data, b*batch_size, (b+1)*batch_size)
        acc+=b_acc/nbatch
        if b%100==0:
            print('-batch number:', b,  ' '*(len(str(nbatch))-len(str(b))), '-- IOU accuracy:', b_acc, '-- time interval:', datetime.now()-btime, '-')
            btime = datetime.now()
    print('\nIOU accuracy:', acc, '\nTime interval:', datetime.now()-stime)
    return acc

In [9]:
def augment_data(data, ratio=0.5, addNoise=False):
        '''
        The function add data points. 
        We pick a query from a random data point,
        and a set of bbox from a different random point and we join them
        to build a new data point. The label distribution of the new data point will 
        uniform, that is, all labels will have equal probability. 
        
        
        Params:
            data: Each entry is in this list has the following structure:
                  [query indexes, [bounding box vector, bounding box spaital features], ..., 
                  [bounding box vector, bounding box spaital features], list, label distribution]
                  *Queries and images are padded.
                                                
        Returns: augmented data
            
                        
        '''
                          
        q_idx = np.random.choice(range(len(data)), int(len(data)*ratio), replace=False)
        im_idx = np.random.choice([i for i in range(len(data)) if i not in q_idx], int(len(data)*ratio))
        for i in range(len(q_idx)):
            q, im = data[q_idx[i]][0], data[im_idx[i]][1:-1]
            item = [q]
            for im_tmp in im:
                item.append(im_tmp)
            item.append(-len(im))
            data.append(item)

# ALSTM

<h4>
<p>The cell build 2 LSTM cells, Bcell and Acell and work as follow:</p>
<ol> 
<li>We run B's cells with the true input getting the un_edited state and and feed it to A's cell. We also run it on the edited input, 'unk' vector.<br>If use_worsAttn=True we add attention on the images vectors for each word.</li><br>
<li>We run A's cells, A's input are B's un-edited state, the reward for editing a word and B's loss having no edited words. </li><br>
<li>A's output is then goes throw a transformation which yields two values, one for editing a word and another for not.<br>If the the value for edit the word is higher then we pass B's edited state to the next time step, else we pass B's un_edit state.</li><br>
</ol></h4>

In [10]:
class ALSTM(tf.nn.rnn_cell.LSTMCell):
    def __init__(self, 
                 batch_size, 
                 num_units, 
                 
                 # Size of A's attention vector.
                 words_attn_dim, 
                 # B's outputs (for each time step).
                 words_attn_states, 
                 # Inicates whether attentionvec is a padding (0) or not (1).
                 words_attn_idx, 
                 
                 # Size of B's attention vector ([image vector, spital features] size) .
                 img_attn_dim, 
                 # B's outputs (for each time step).
                 img_attn_states, 
                 
                 
                 # Inicates whether attentionvec is a padding (0) or not (1).
                 img_attn_idx, 
                 
                 unk, #'unk' word vector
                 
                 # Probabilty for edit a word in rnn, 
                 # when decision are taken randomly.
                 rnn_editProb, 
                 
                 
                 # Whehter to edit the query or not.
                 isEdit, 
                 
                 # Whether B uses attention  or not.
                 use_wordAttn,
                 
                 # If True add noise instead of using 'unk'
                 useNoise=False,
                 
                 # If useNoise is true word vec = alpha*word_vector+(1-alpha)*noise
                 alpha=.3,
                 
                 # when isEdit=True, whether or not to use A's 
                 # output inorder to edit or to do it randomly.
                 activateA=False,
                 
                 #this holds A's rewards and B's losses to
                 # be add to A's feature vectors.
                 reward_loss=None,
                 state_is_tuple=True,):
        
        
        # When useing A, the cell state will contain the concatenation 
        # of both B and A states. Therefore we set the unit number to be
        # 2*(A and B unit size).
        super().__init__(2*num_units, state_is_tuple=state_is_tuple)
    
        self.words_attn_states = words_attn_states
        self.words_attn_idx = words_attn_idx
        self.words_attn_dim = words_attn_dim
        
        self.img_attn_states = img_attn_states
        self.img_attn_idx = img_attn_idx
        self.img_attn_dim = img_attn_dim
        
        self.num_units = num_units
        self.batch_size = batch_size
        self.unk = unk 
        self.isEdit = isEdit
        self.activateA = activateA
        self.use_wordAttn=use_wordAttn
        
        
        self.Acell = tf.nn.rnn_cell.LSTMCell(self.num_units, state_is_tuple=True)
        self.Bcell = tf.nn.rnn_cell.LSTMCell(self.num_units, state_is_tuple=True)
        
        self.rnn_editProb = rnn_editProb
        self.reward_loss = reward_loss
        self.useNoise=useNoise
        self.alpha=alpha
        

    def call(self, inputs, state):
        '''
        Params:
            inputs: word embadding.
            state:  [B's state form privious state, A's state form privious state]
        '''
        # takse B's state from state[:self.num_units]
        Bstate_c = tf.slice(state[0], [0, 0], [-1, self.num_units])
        Bstate_h = tf.slice(state[1], [0, 0], [-1, self.num_units])
        self.Bstate =  tf.nn.rnn_cell.LSTMStateTuple(c=Bstate_c, h=Bstate_h)
         
        # If B's cell uses attention
        def g1():
            new_input = self.attention(Bstate_h, self.img_attn_states, self.img_attn_dim, self.img_attn_idx)
            return self.Bcell(inputs, self.Bstate, 'Bcell')
        
        # If B's cell does not use attention
        def g2():
            return self.Bcell(inputs, self.Bstate, 'Bcell')
        
        # Run B's cell on un-edited inputs.
        Boutputs, Bnew_state = tf.cond(self.use_wordAttn, g1, g2)
        
        
        # If isEdit==True 
        def f1(): 
            # takse A's state from state[self.num_units: 2*self.num_units]
            Astate_c = tf.slice(state[0], [0, self.num_units], [-1, self.num_units])
            Astate_h = tf.slice(state[1], [0, self.num_units], [-1, self.num_units])
            self.Astate =  tf.nn.rnn_cell.LSTMStateTuple(c=Astate_c, h=Astate_h)
            
            if self.useNoise: # just add noise to the edited words
                new_unk_vecs = self.alpha*inputs + (1-alpha)*tf.random_normal(shape=inputs.get_shape(), stddev=0.1)
            else: # change the edited words by 'unk'
                unk_vecs = tf.concat([self.unk for _ in range(self.batch_size)], 0) # shape: self.batch_size x 1 x embed_size
                new_unk_vecs = tf.squeeze(unk_vecs) # shape: self.batch_size x embed_size
            
            # run B's cell with unk_batch
            edit_output, edit_new_state = self.Bcell(new_unk_vecs, self.Bstate, 'Bcell')  
            out1, state1 = self.runCell(Boutputs, Bnew_state, edit_new_state)
            return out1, state1
        
        def f2(): 
            outs = tf.concat([Boutputs, tf.zeros((self.batch_size, self.num_units))], 1)
            new_state_c = tf.concat([Bnew_state[0], tf.zeros_like(Bnew_state[0])], 1)
            new_state_h = tf.concat([Bnew_state[1], tf.zeros_like(Bnew_state[1])], 1)
            return outs, tf.nn.rnn_cell.LSTMStateTuple(c=new_state_c, h=new_state_h)
        
        new_output, new_state = tf.cond(self.isEdit, f1, f2)
        
        return new_output, new_state
    
    def runCell(self, Boutputs, Bnew_state, edit_new_state):
        '''
        Run B's cell after editing.
        
        params:
            Boutputs: output vector without editing.
            Bnew_state: state vector without editing.
            edit_new_state: state vector after editing the input word to 'unk'. 
        '''
        
        with tf.variable_scope('runcell'):
            # get action values according to B's hidden state
            Aout, Anew_state, actions_vals = self.action_vals(Boutputs) 
            
            def f1(): 
                '''
                If self.activateA==True, edit word if actions_vals[0]<actions_vals[1].
                Note: if we edit the word cond=1, else cond=0.
                '''
                a1, a2 = tf.split(value=actions_vals, num_or_size_splits=2, axis=1)
                cond = tf.less(a1, a2)
                return cond

            def f2(): 
                '''
                If self.activateA==False, choose randomly whether to edit a word.
                We edit a word with 'rnn_editProb' probability (~U[0,1]).
                Note: if we edit the word cond=1, else cond=0.
                '''
                rand = tf.multinomial(tf.log([[self.rnn_editProb, 1-self.rnn_editProb]]), self.batch_size)
                cond = tf.less(tf.transpose(rand), 1) 
                return cond

            # a list of A's decisions for each batch.  1->edit, 0-> do not edit.
            cond = tf.cast(tf.cond(self.activateA, f1, f2), tf.float32)

            # We'd like to know the action values and decision for each word,
            # therefore theses info are placed on the first 3 dimensions of the 
            # output vector. Note that this vector is not passed to the 
            # next tiee step so it won't affect the model. 
            outs = tf.concat([actions_vals, cond], 1)

            # B's i state is replaced by the edited state if cond[i]=1 (i =1, 2, ..., batch_size)
            new_edit_state_c = (1-cond)*Bnew_state[0] + cond*edit_new_state[0]
            new_edit_state_h = (1-cond)*Bnew_state[1] + cond*edit_new_state[1]


            new_outs = tf.concat([outs, tf.zeros((self.batch_size, 2*self.num_units-3))], 1)
            new_state_c = tf.concat([new_edit_state_c, Anew_state[0]], 1)
            new_state_h = tf.concat([new_edit_state_h, Anew_state[1]], 1)
            return new_outs, tf.nn.rnn_cell.LSTMStateTuple(c=new_state_c, h=new_state_h)
    
    
    def action_vals(self, Boutputs):
        '''
        Get values for editing/not editing the input word.
        
        Params:
            Boutputs: output vector without editing.
            
        Returns vals where:
            Aout: A's cell output
            Anew_state: A's cell state
            vals: Tensor where vals[0] is the value for not editing the word 
                    and vals[1] is the value for editing the word.
        '''
        
        with tf.variable_scope('action_vals') as scope:
            Aattn = self.attention(self.Astate[1], self.words_attn_states, self.words_attn_dim, self.words_attn_idx)
            
            # A's input: [input, B's output, attntion state, reward, B's loss with no edits]
            Anew_inputs = tf.concat([Boutputs, Aattn, self.reward_loss], 1)
            
            Aout, Anew_state = self.Acell(Anew_inputs, self.Astate, 'Acell')
            vals = tf.nn.relu(self.linear(Aout, 2))

            # Save the variables that only A uses. 
            # These variables will be trained separately from B's model.
            self.Avars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope.name)
            
            return Aout, Anew_state, vals
            
        
    def linear(self, inputs, output_dim, scope='linear', bias=True, reuse=False):
        with tf.variable_scope(scope, reuse=False):
            W = tf.get_variable('W', initializer=tf.random_uniform_initializer(maxval=1., minval=-1.),
                                shape=(inputs.get_shape()[-1], output_dim))
            if bias:
                b = tf.get_variable('b', initializer=tf.constant_initializer(0.1),
                               shape=[1, output_dim])
                return tf.matmul(inputs, W) + b

        return tf.matmul(inputs, W)
    
    
    def attention(self, state, attn_states, attn_dim, attn_idx, relu=False):
        '''
        Attention mechanism (see https://arxiv.org/pdf/1409.0473.pdf)
        
        state: State from previous time step.
        attn_states: Attetntion states. 
                     Tensor of shape (batch_size x max([len(attention_vectors[i]) for i in range(bach_size)]) x attn_dim)
        attn_dim: Attention vector size.
        attn_idx,: Tensor used for masking of shape (batch_size x max([len(attention_vectors[i]) for i in range(bach_size)]). 
                   attn_idx[i, j]=1 if the j's attention vcctior of sample i  is not padding, else its equat to 0.
        '''
        
        self.attn_length = tf.shape(attn_states)[1]  
        
        # Computing... hidden_attn = W*v_att (use tf.nn.conv2d for efficiency)
        attn_vecs = tf.reshape(attn_states, [self.batch_size, self.attn_length, 1, attn_dim])
        W = tf.get_variable("attn_W", [1, 1, attn_dim, self.num_units])
        hidden_attn = tf.nn.conv2d(attn_vecs, W, [1, 1, 1, 1], "SAME")

        # Computing... hidden_s = U*v_state
        hidden_s = tf.reshape(
            self.linear(
                tf.cast(state, tf.float32), output_dim=self.num_units, scope='hidden_s_linear'), [-1, 1, 1,  self.num_units], name='hidden_s')

        # Computing alpha
        v = tf.get_variable("attn_v", [self.num_units])
        if relu:
            logits = tf.reduce_sum(v * tf.nn.relu(hidden_attn + hidden_s), [2, 3])
        else:
            logits = tf.reduce_sum(v * tf.nn.tanh(hidden_attn + hidden_s), [2, 3])

        # Masked softmax
        max_logits = tf.reduce_max(logits, axis=-1)
        masked_logits = tf.exp(logits-tf.expand_dims(max_logits, axis=1))*attn_idx
        alpha = masked_logits/tf.reduce_sum(masked_logits, axis=-1, keep_dims=True)

        a = tf.reduce_sum(tf.reshape(alpha, [-1, self.attn_length, 1, 1]) * attn_vecs, [1, 2])
        b = tf.contrib.layers.fully_connected(a, num_outputs=self.num_units, 
                                                normalizer_fn=tf.contrib.layers.batch_norm, scope='proj')

        return b

In [11]:
class Model():
    def __init__(self,
                 batch_size, 
                 num_hidden, 
                 embed_size,
                 
                 #Image's vector size.
                 img_dims, 
                 
                 #Spaital features length.
                 bbox_dims, 
                 
                 #Probabilty for edit a word in rnn, when decision are taken randomly.
                 rnn_editProb,  
                 vocab, 
                 lr, #  B's learning rate.
                 decay_steps, 
                 decay_rate, 
                 
                # coefficient to multiply the reward by when editing a word.
                 edit_reward,
                 
                 # A's leanring rate = B's learning rate x coefAlr.
                 coefAlr,
                 
                 # whether to use bach normaliztion for the last attention layer
                 bnorm,
                 useBidirectionalRnn=False):
        
        self.batch_size = batch_size
        self.img_dims = img_dims
        self.bbox_dims = bbox_dims 
        self.num_hidden = num_hidden
        self.embed_size = embed_size
        self.vocab = vocab
        L2Reg = 0.0005 # L2 regulizerr coefficient
        self.lr = lr


        self.queries = tf.placeholder(tf.int32, [None, None], name='queries')
        self.img  = tf.placeholder(tf.float32, [None, None, self.img_dims], name='img')# VGG output vectors
        self.bboxes = tf.placeholder(tf.float32, [None, None, self.bbox_dims], name='bboxes')# spatial bbox's features.
        
        # attn_idx: inicates whether attention box is a dummy (0) or not (1).
        self.attn_idx = tf.placeholder(tf.float32, [None, None], name='attn_idx')
        
        self.labels = tf.placeholder(tf.float32, [None, None], name='labels')
        self.isEdit = tf.placeholder(tf.bool, name='isEdit') # whehter to edit the query or not.
        
        # when isEdit=True, whether to use A's output in order to edit or to do it randomly.
        self.activateA = tf.placeholder(tf.bool, name='activateA') 
        
       
        # this holds  A's rewards and B losses to be add to A's feature vectors.
        self.reward_loss = tf.placeholder(tf.float32, [None,2], name='rewards_loss')
        
        # Dropout ratio for rnn's inputs and outpouts
        self.dropout_in = tf.placeholder(tf.float32, name='dropoutIn_holder')
        self.dropout_out = tf.placeholder(tf.float32, name='dropoutOut_holder')
        
        # Dropout ratio for images vector (for the final attention layer before the loss function)
        self.dropout_img = tf.placeholder(tf.float32, name='dropoutImg_holder')
        
        # Whether B uses words level attentinon 
        self.use_wordAttn = tf.placeholder(tf.bool, name='use_worsAttn_holder')
        
        # B outputs vectors (with no words edits), These are A's attention vectors
        # which it uses to decide whter to edit a word.
        self.Aattn_vecs = tf.placeholder(tf.float32, [None, None, None], name='Aattn_vecs_holder')    
        self.unk = tf.constant([[vocab['<unk>']]], tf.int32)

        self.queries_lens = self.length(self.queries) # list of all the lengths  of the batch's queriey 
        
        # Concatinate images vectors and their spaital features. 
        # These vectors wlll be used for attenionn when 
        # we calculate the loss function.
        attn_vecs = tf.concat([self.img, self.bboxes], 2) 
        voc_size = len(self.vocab)

        with tf.variable_scope('embed'):
            embed = tf.get_variable('embed', shape=[voc_size, self.embed_size], 
                                    initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
            embed_queries = tf.nn.embedding_lookup(embed, self.queries, name='embed_queries')

        with tf.variable_scope('rnn'):
            Aattn_idx = tf.cast(tf.abs(tf.sign(self.queries)), tf.float32)
            
            
            cell_tmp = ALSTM(num_units=self.num_hidden, 
                            words_attn_dim=self.num_hidden, 
                            words_attn_states=self.Aattn_vecs, 
                            words_attn_idx=Aattn_idx,
                            img_attn_dim=self.img_dims+self.bbox_dims,
                            img_attn_states=attn_vecs,
                            img_attn_idx=self.attn_idx,
                            batch_size=self.batch_size, 
                            unk=tf.nn.embedding_lookup(embed, self.unk), rnn_editProb=rnn_editProb,
                            activateA=self.activateA, isEdit=self.isEdit,
                            reward_loss=self.reward_loss, use_wordAttn=self.use_wordAttn)
            
            cell = tf.nn.rnn_cell.DropoutWrapper(
                cell_tmp, input_keep_prob=self.dropout_in, output_keep_prob=self.dropout_out)
            
            if useBidirectionalRnn:
                self.outputs, self.last_states = tf.nn.bidirectional_dynamic_rnn(
                    cell_fw=cell,
                    cell_bw=cell,
                    dtype=tf.float32,
                    sequence_length=self.queries_lens,
                    inputs=embed_queries)
            else:
                self.outputs, self.last_states = tf.nn.dynamic_rnn(
                    cell=cell,
                    dtype=tf.float32,
                    sequence_length=self.queries_lens,
                    inputs=embed_queries)

        Avars = cell_tmp.Avars
        self.Avars = {var.name:var for var in Avars}
        # self.values[0]=value for not editing, self.values[1]=value for editing
        self.values = tf.slice(self.outputs, [0,0,0], [-1,-1,2])
        Bstate = tf.slice(self.last_states[1], [0,0], [-1, self.num_hidden])  
        self.scores = self.attention(Bstate, bnorm) 
        
                          
        # Cross entophy loss for each of the queries in the batch.
        self.B_ce = -tf.reduce_sum(
                        self.labels*tf.log(self.scores+0.00000001)+
                            (1-self.labels)*tf.log((1-self.scores)+0.00000001), 
                            axis=-1)

        # A's decision for each word.
        self.idx = tf.squeeze(tf.slice(self.outputs, [0,0,2], [-1,-1,1]))

        self.edit_num = tf.reduce_mean(tf.reduce_sum(
            tf.cast(self.idx, tf.float32)*tf.expand_dims(
                    1/tf.cast(self.queries_lens, tf.float32), axis=1), axis=1))

        # After running A for the first time, we get A's decisions and their values.
        # we then calulate the following tensors:
        # actions_idx[j,i] = 1 if the word i in query j was edited or 0 otherwise.     
        # bell_vall holds the values for each decision by the bellman function. 
                          
        #  self.actions_idx will be used to get the values of the action that were taken by A.                 
        self.actions_idx = tf.placeholder(shape=[None, None], dtype=tf.int32, name="actions_idx")
        self.bell_val = tf.placeholder(shape=[None, None], dtype=tf.float32, name="bell_val")
        
        # pyrite_val: holds the values for each of A's decisions, claculated by A. 
        # see https://en.wikipedia.org/wiki/Pyrite.
        self.pyrite_val = tf.reshape(tf.gather_nd(tf.reshape(self.values, (-1,2)), self.actions_idx), (self.batch_size, -1))

        # RMSE loss
        self.A_loss = tf.reduce_mean(tf.reduce_sum(
            tf.sqrt(tf.square((self.bell_val-self.pyrite_val+0.000001)*tf.cast(
                tf.sign(tf.abs(self.queries)), tf.float32)))/tf.expand_dims(
                                                tf.cast(self.queries_lens, tf.float32), axis=1), axis=-1))

        self.B_loss = tf.reduce_mean(self.B_ce)

        ##############
        # Optimizers #
        ##############

        starter_learning_rate = self.lr
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        self.learning_rate = tf.train.exponential_decay(starter_learning_rate, self.global_step,
                                                       decay_steps=decay_steps, decay_rate=decay_rate, staircase=True)

        # Train only A variables 
        self.A_optimizer =  tf.train.GradientDescentOptimizer(
                learning_rate=coefAlr*self.learning_rate).minimize(self.A_loss, var_list=Avars)  

        # Train only B variables 
        Bvars = [var for var in tf.trainable_variables() if var not in Avars]
        self.B_optimizer =  tf.train.GradientDescentOptimizer(
                    learning_rate=self.learning_rate).minimize(self.B_loss, global_step=self.global_step, var_list=Bvars)  

        if not os.path.exists(params_dir):
                os.makedirs(params_dir)
        self.saver = tf.train.Saver()

        
    def length(self, seq):
        '''
        Retruns real lengths (before addings) of all queries in seq  .
        '''
        return tf.cast(tf.reduce_sum(tf.sign(tf.abs(seq)), reduction_indices=1), tf.int32)
       

    def linear(self, inputs, output_dim, scope='linear', bias=True, reuse=False):

        with tf.variable_scope(scope, reuse=reuse):
            W = tf.get_variable('W', initializer=tf.contrib.layers.xavier_initializer(),
                                shape=(inputs.get_shape()[-1], output_dim))
            if bias:
                b = tf.get_variable('b', initializer=tf.constant_initializer(0.1),
                               shape=[1, output_dim])
                return tf.matmul(inputs, W) + b
            
            return tf.matmul(inputs, W)

            
    def attention(self, q_embed, bnorm):
        '''
        Given B's output vector, calculate the attention over all the query's bounding boxes vectors.
        The  bounding boxe with the highest attention score will be chosen as the correct bounding box.
        
        Params:
            q_embed: Tensor of shape (batch size x num_hidden)B's outputs. 
            bnorm: Boolean tensor. Whether to use batch normalization.
            
        Returns:
            probs: Tensor of shape (batch_size x max bbox number for query).
                   Attention score for each bbox.
        '''
        # concatenate img vectors with with spaical features
        attn_vecs = tf.concat([self.img, self.bboxes], 2)
        Urnn = q_embed # B's outputs. shape: (batch size x num_hidden)
        Uatt = tf.nn.dropout(attn_vecs, keep_prob=self.dropout_img) # Attention vectors. shape: (batch size x max bbox number for query x attention vector size)
        
        # If bnorm is true use batch normalization
        if bnorm:    
            normalizer_fn=tf.contrib.layers.batch_norm
        else:
            normalizer_fn=None
                          
        with tf.variable_scope('l1'):
            b = tf.get_variable(
                'b1', initializer=tf.constant_initializer(0.1), shape=[1, self.num_hidden])
            
            context = tf.get_variable(
                'context', initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1), shape=[self.num_hidden, 1])
            
            
            rnn_out = tf.contrib.layers.fully_connected(
                inputs=Urnn, 
                num_outputs=self.num_hidden, 
                activation_fn=None, 
                normalizer_fn=normalizer_fn
            ) 
            
            attn_out = tf.contrib.layers.fully_connected(
                inputs=Uatt, 
                num_outputs=self.num_hidden, 
                activation_fn=None, 
                normalizer_fn=normalizer_fn
            )
            
        out = tf.nn.relu(tf.expand_dims(rnn_out, 1) + attn_out + b)
        logits = tf.reshape(tf.matmul(tf.reshape(out, (-1, tf.shape(out)[-1])),  context), (tf.shape(out)[0], -1))
                          
        # Calculate logits's masked softmax (we use self.attn_idx to mas
        max_logits = tf.reduce_max(logits, axis=-1)
        masked_logits = tf.exp(logits-tf.expand_dims(max_logits, axis=1))*self.attn_idx
        probs = self.attn_idx*masked_logits/tf.reduce_sum(masked_logits, axis=-1, keep_dims=True)

        return probs
  
        
    def q_padding(self, seq, max_length):
        '''
        Pad  seq with vocab['<pad>'] (0) to max_length length.
        '''                  
        return seq + [self.vocab['<pad>']]*(max_length-len(seq))

    
    def build_data(self, data, start, end, addNoise=False):
        '''
        Build batch.
        ------------
        
        Params:
            data: each entry in this list has the following structure:
                  [query indexes, [bounding box vector (VGG), bounding box spaital features], ..., 
                  [bounding box vector (VGG), bounding box spaital features], index of the true label]
                  
            start/end: batch data is built from data[start:end]
            
        Returns:
            attn_idx: attn_idx[i, j]=1 if the j'th bbox in the i'th query is not padding, else equals to 0. 
            
            padded_queries: list of queries, padded to the length of the longest query in the batch.
                            Note: vocab['pad']=0
                            
            padded_im: list of bounding boxes vectors, padded to the maximum number of bbox per query.
                       Note: padded vector is vector of zeros. 
                            
            padded_bbox: list of bounding boxes spatial features, padded to the maximum number of bbox per query.
                         Note: padded vector is vector of zeros.  
        
            onehot_labels: onehot_labels[i][j]=1 if j is the true bbox for query i, else  onehot_labels[i][j]=0
            
            addNoise: Boolean. Whether to add normal noise to the images.
                        
        '''
                          
        qlen = max([len(data[i][0]) for i in range(start, end)]) # Length fo the longest query
        imlen = max([len(data[i]) for i in range(start, end)])-2 # Maximum number of bbox per query.
        padded_queries, padded_im, padded_bbox, attn_idx = [], [], [], []
        
        # Build one hot labels from the labels index, given in the data.                  
        labels = [item[-1] for item in data[start:end]] #data[i][-1]=index of the true bbox of query i
        dist_labels = np.zeros((end-start, imlen)) #label distribution
        
        # Real data points
        dist_labels[[i for i in np.arange(end-start) if labels[i]>0], [l for l in labels if l>0]]=1
        
        # augmented data points
        for i in np.arange(end-start):
            if labels[i]<0:
                dist_labels[i] = [-1/labels[i] for _ in range(-labels[i])]+[0. for _ in range(imlen+labels[i])]
                          
        im_dim, bbox_dim = data[0][1][0].shape[1], data[0][1][1].shape[1]
        for i in range(start, end):
            padded_queries.append(self.q_padding(data[i][0], qlen))
            
            attn_idx.append([1 for _ in range(len(data[i])-2)]+[0 for _ in range(imlen-(len(data[i])-2))])
            
            padded_im.append(np.concatenate([data[i][j][0] for j in range(1, len(data[i])-1)] + 
                                       [np.full((imlen-(len(data[i])-2), im_dim), vocab['<pad>'], dtype=np.float32)], axis=0))
            
            padded_bbox.append(np.concatenate([data[i][j][1] for j in range(1, len(data[i])-1)] + 
                                       [np.full((imlen-(len(data[i])-2),bbox_dim), vocab['<pad>'], dtype=np.float32)], axis=0))
           
        if addNoise:
            padded_im+=(padded_im+np.random.normal(0, .1, np.array(padded_im).shape))*np.expand_dims(attn_idx, 2)
        else:
            padded_im=np.array(padded_im)
            
        return np.array(attn_idx), np.array(padded_queries, dtype=np.int32), padded_im, np.array(padded_bbox), np.array(dist_labels)
            
   
    def ground(self, data=None, start=None, end=None, sess=None, feed_dict = None, isEdit=True):
        '''
        Given a query and a list of bboxes, the function returns the index of the referred bbox.
        '''
        isSess = (sess==None)
        if isSess:
            sess = tf.Session()
        with sess.as_default():
            if isSess:
                tf.global_variables_initializer().run()
                ckpt = tf.train.get_checkpoint_state(params_dir)
                if ckpt and ckpt.model_checkpoint_path:
                    self.saver.restore(sess, ckpt.model_checkpoint_path) # restore all variables
                else:
                    print('Initializing variables')
            if feed_dict is None:
                attn_idx, padded_queries, padded_im, padded_bbox, labels = self.build_data(data, start, end)
                feed_dict = {
                        self.queries:padded_queries,
                        self.img:padded_im,
                        self.bboxes:padded_bbox,
                        self.labels:labels,
                        self.attn_idx:attn_idx
                    }
            feed_dict[self.isEdit] = isEdit
            feed_dict[self.dropout_in]=1.
            feed_dict[self.dropout_out]=1.
            feed_dict[self.dropout_img]=1.
            scores = sess.run(self.scores, feed_dict=feed_dict) # get score for each bbox

        return np.argmax(scores, axis=1), np.argmax(feed_dict[self.labels], axis=1)
        
        
    def iou_accuracy(self, data, start, end, sess=None, feed_dict = None, threshold=0.5, test=False, isEdit=True):
        '''
        Calculate the IOU score between the Model bbox and the true bbox.
        ''' 
                          
        # Get score for each bbox (labels) and th true bbox index (gt_idx)                  
        if feed_dict is None:
            labels, gt_idx = self.ground(data, start, end, sess=sess, feed_dict=feed_dict, isEdit=isEdit)
        else: labels, gt_idx = self.ground(sess=sess, feed_dict=feed_dict, isEdit=isEdit)
        acc = 0
        
        for i in range(start, end):
            gt = data[i][gt_idx[i-start]+1][1][0] # ground truth bbox
            crops = np.expand_dims(data[i][labels[i-start]+1][1][0], axis=0) #Model chosen bbox
            acc += (retriever.compute_iou(crops, gt)[0]>threshold) #IOU for the i sample.
            
        return acc/(end-start)
        
    def accuracy(self, data=None, start=None, end=None, sess=None, feed_dict = None, isEdit=True):
        isSess = (sess==None)
        if isSess:
            print('Building sess')
            sess = tf.Session()
        with sess.as_default():
            if isSess:
                print('Building sess used')
                tf.global_variables_initializer().run()
                ckpt = tf.train.get_checkpoint_state(params_dir)
                if ckpt and ckpt.model_checkpoint_path:
                    print('3')
                    self.saver.restore(sess, ckpt.model_checkpoint_path) # restore all variables
                else:
                    print('Initializing variables')
            if feed_dict is None:
                print('Building feed_dict')
                attn_idx, padded_queries, padded_im, padded_bbox, labels = self.build_data(data, start, end)
                feed_dict = {
                        self.queries:padded_queries,
                        self.img:padded_im,
                        self.bboxes:padded_bbox,
                        self.attn_idx:attn_idx,
                        self.labels:labels,
                    }
                
            feed_dict[self.isEdit] = isEdit
            feed_dict[self.dropout_in]=1.
            feed_dict[self.dropout_out]=1.
            feed_dict[self.dropout_img]=1.
            scores = sess.run(self.scores, feed_dict=feed_dict)
            acc = sum(np.equal(np.argmax(scores, axis=1), np.argmax(feed_dict[self.labels], axis=1))/len(feed_dict[self.labels]))

                    
        return acc
    
    def discount_rewards(self, r, last_r, gamma=1.0):
        """ 
        take 1D float array of rewards and compute discounted reward 
        using bellman function.
        
        params:
            r: r[i,j]=1 reward of action A did on word j at query i.
            last_r: B's loss for query i with no edits. This is the final reward.
            gamma: discount factor.
        """
                          
        discounted_r = np.zeros(r.shape)
        running_add = last_r # B loss
        discounted_r = [i for i in range(r.shape[-1])]
        for t in reversed(range(0, r.shape[-1])):
            running_add = running_add * gamma + r[:,t]
            discounted_r[t] = running_add
        return np.array(discounted_r).T
        
    def train(self, trn_data, tst_data, ephocs_num, edit_reward, startA=3, 
              activation_ephoc=10, muteB=3, start_ephoc=0, dropout_in=1.,
              dropout_out=1., dropout_img=1., editProb=0.5, max_activateAProb=0.8, 
              activateAProb = 0.5, use_wordAttn=False, addNoise=False):
                          
        '''
        Params:
             trn_data: list, train set. 
             
             tst_data: list, test set. 
             
             ephocs_num: number of ephocs
             
             start_ephoc: number of first ephoc.
             
             edit_reward: int, coefficient to multiply the reward by when editing a word.
             
             startA: int, Start competition only at ephoc # startA.
             
             activation_ephoc: at ephoc numer "activation_ephoc", A will be activate.
                               That is, for (activation_ephoc-startA) number of ephocs, 
                               A will chooce an action randomly.
             
            muteB: After A starts, for each ephoc which A & B trains, 
                   only A will be trained for this amount of ephocs.
                   
            editProb: robabilty for editing a query.
            
            activateAProb: when running A, we can choos an action randomly or taknig A decision. 
                            This is the starting probabilty for NOT choocing an action ranomdly.
            
            max_activateAProb: Final probabilty for NOT choocing an action ranomdly.
            
            dropout_in: dropout ratio of B's rnn inputs.
            
            dropout_output: dropout ratio of B's rnn output.
            
            dropout_img: dropout ratio of images vectors before the last attention layer .
            
            addNoise: Boolean. Whether to add normal noise to the images (see build_data).
                               
        '''                  
        
        trn_nbatch = len(trn_data)//self.batch_size
        tst_nbatch = len(tst_data)//self.batch_size
        print('# Train set size:', len(trn_data))
        print('# Training batches:', trn_nbatch)
        print('# Test set size:', len(tst_data))
        print('# Testing batches:', tst_nbatch)
        self.test_res, self.train_res = [], [] #list to hold accuracy of test set
        sess = tf.Session()
        with sess.as_default():
            tf.global_variables_initializer().run()
            ckpt = tf.train.get_checkpoint_state(params_dir)
            if ckpt and ckpt.model_checkpoint_path:
                print('Loading parameters from', ckpt.model_checkpoint_path)
                self.saver.restore(sess, ckpt.model_checkpoint_path) # restore all variables
            else:
                print('Initializing variables')
                
            for ephoc in range(start_ephoc, ephocs_num):
                startTime = datetime.now().replace(microsecond=0)
                          
                # Train B only after competition start (ephoc<startA). 
                # Each time we train B, we train only B for muteB ephocs.
                if startA>0:
                    toTrainB = ephoc<startA or (ephoc-(startA-1))%(muteB+1)==0 # Whether to train only B 
                else:
                    toTrainB = ephoc==0 or ephoc%(muteB+1)==0 # Whether to train only B 
                    
                # Every 2 times we train B and A (+ only A for muteB ephocs), the
                # probability for choosing an action using A and not randomly, increases by 1.1.
                if activateAProb>0 and activateAProb<max_activateAProb and muteB>0 and (ephoc-activation_ephoc)%((muteB+1)*6)==0 and ephoc>activation_ephoc:
                    activateAProb=activateAProb*1.1
                        
                itrInEphoc=1 # When we train B, we show B results without editing (on train set)
                if activateAProb>0 and ephoc>=startA and toTrainB:
                    itrInEphoc=2
                    
                print('='*50,'\nTrain, ephoc:',ephoc, '; Training B:', toTrainB)
                np.random.shuffle(trn_data)
                for ep in range(itrInEphoc):
                    trn_ce_loss, A_trn_loss, B_trn_loss = 0, 0, 0
                    trn_acc, trn_iou, edits_count = 0, 0, 0
                    tst_ce_loss, tst_loss, tst_acc, tst_iou = 0, 0, 0, 0
                    if ep==1:
                        print('No Edit')
                        print('ooooooo')
                    edit_num = 0 # number of edited words
                    editRount_count = 0 # number of iterations with competition.
                    for b in range(trn_nbatch):
                        if ep==1:
                            '''
                            If ep=1, we're in round 2 in which we only show B's 
                            results with out editing (on the train set).
                            '''
                            isEdit=False
                            isActivateA=False
                        else:
                            isEdit = ephoc>=startA and (toTrainB==False or np.random.rand(1)[0]<editProb)
                            # Whether to choose an action using A or randomly.
                            isActivateA = (toTrainB==True and activateAProb>0.) or (isEdit and ephoc>=activation_ephoc and np.random.rand(1)[0]<activateAProb)
            
                        attn_idx, padded_queries, padded_im, padded_bbox, labels = self.build_data(trn_data, 
                                                                                            b*self.batch_size, (b+1)*self.batch_size, addNoise=addNoise)
                
                        reward_loss = np.array([[0., 0.] for _ in range(self.batch_size)]) # dummy holder 
                    

                        feed_dict = {
                            self.queries:padded_queries,
                            self.img:padded_im,
                            self.bboxes:padded_bbox,
                            self.attn_idx:attn_idx,
                            self.labels: labels,
                            self.unk:np.array([[vocab['<unk>']]]),
                            self.isEdit:isEdit,
                            self.activateA:isActivateA,
                            self.reward_loss:reward_loss,
                            self.Aattn_vecs:[[[]]],# dummy holder 
                            self.dropout_in:dropout_in,
                            self.dropout_out:dropout_out,
                            self.dropout_img:dropout_img,
                            self.use_wordAttn:use_wordAttn
                        }
                        
                        
                        if isEdit:
                            editRount_count+=1
                          
                            '''We first run B with no edit in order to get it's loss and outputs.'''
                          
                            feed_dict[self.isEdit] = False
                            queries_lens, B_ce, outputs = sess.run([self.queries_lens, self.B_ce, self.outputs], feed_dict=feed_dict)
                            feed_dict[self.isEdit] = True
                            
                            # B_ce contain the loss for each query.
                            # We scale all the losses to have mean 1 and std 0.5.
                            Bce_min = min(B_ce)
                            Bce_max = max(B_ce)
                            Bce_scale = (B_ce-Bce_min)/(Bce_max-Bce_min)
                            s = np.std(Bce_scale)
                            BCE = Bce_scale/(2.*s)
                            BCE = 1 + BCE - np.mean(BCE)
                          
                            # reward for edit a word = edit_reward*BCE/queries_lens.
                            # A's get the reward and loss per query  as 2 of its features.
                            rewards_loss = np.concatenate(
                                [np.expand_dims(edit_reward*BCE/queries_lens, 1), np.expand_dims(BCE, 1)], 1)
                            
                            feed_dict[self.reward_loss]=rewards_loss
                            
                            # A's attention vectors are B's outputs with no edited words.
                            Aattn_vecs = outputs[:,:,:self.num_hidden]
                            feed_dict[self.Aattn_vecs]=Aattn_vecs
                           
                            '''
                            We then run B with A edits. 
                            We get idx (A's decision [0,1] for each words), edit_num: the ratio
                            of edited words and B_ce, B_loss: loss poer query and B_vce mean.
                            '''
                            feed_dict[self.isEdit] = False
                            idx, B_ce, B_loss, edit_num = sess.run(
                                [self.idx, self.B_ce, self.B_loss, self.edit_num], 
                                 feed_dict=feed_dict)
                            
                            # Get rewurd per word.
                            # If word i in query j was edited idx[j,i]=1 --> rewards[i,j] = 1*reward
                            # If word i in query j was not edited idx[j,i]=0 --> rewards[i,j] = 0*reward
                            rewards = idx*np.expand_dims(
                                edit_reward*BCE/queries_lens, axis=1)
                            
                            # Scale new losses
                            Bce_min = min(B_ce)
                            Bce_max = max(B_ce)
                            Bce_scale = (B_ce-Bce_min)/(Bce_max-Bce_min)
                            s = np.std(Bce_scale)
                            BCE1 = Bce_scale/(2.*s)
                            BCE1 = 1 + BCE1 - np.mean(BCE1)
                            
                            feed_dict[self.actions_idx] = np.abs(list(zip(range(
                                        self.batch_size*idx.shape[-1]), idx.reshape(-1))))
                            feed_dict[self.bell_val] = self.discount_rewards(rewards1, BCE1)
                                                         
                            A_loss, _ = sess.run(
                                [self.A_loss, self.A_optimizer], feed_dict=feed_dict)
                            
                            if toTrainB:
                                lr, gs, _ = sess.run(
                                    [self.learning_rate, self.global_step, self.B_optimizer], 
                                    feed_dict=feed_dict)
                
                    
                            acc = self.accuracy(sess=sess, feed_dict=feed_dict, isEdit=isEdit)  
                            iou_acc = self.iou_accuracy(
                                trn_data, b*self.batch_size, (b+1)*self.batch_size, 
                                sess=sess, feed_dict=feed_dict, isEdit=isEdit)

                            trn_acc += acc/trn_nbatch
                            A_trn_loss += A_loss
                            B_trn_loss += B_loss/trn_nbatch
                            trn_iou += iou_acc/trn_nbatch
                            edits_count += edit_num
                            
                            if b%50==0:
                                if toTrainB: 
                                    print('Edit:', feed_dict[self.isEdit], ';A:',isActivateA, 
                                          ';ephoc:',ephoc, ';batch:', b, 
                                          ';gs:', gs, ';lr: %.4f'%(lr), ';B loss: %.2f'%(B_loss), 
                                          ';A loss: %.2f'%(A_loss), ';edits: %.2f'%(edit_num),  ';activateAProb: %.3f'%(activateAProb),
                                          ';acc: %.3f'%(acc), ';iou: %.3f'%(iou_acc), ';time:', datetime.now().replace(microsecond=0)-startTime)    
                                else:
                                    print('Edit:', feed_dict[self.isEdit], ';A:',isActivateA, 
                                          ';ephoc:',ephoc, ';batch:', b, 
                                          ';B loss: %.2f'%(B_loss), ';A loss: %.2f'%(A_loss), 
                                          ';edits: %.2f'%(edit_num), ';activateAProb: %.3f'%(activateAProb), ';acc: %.3f'%(acc),
                                          ';iou: %.3f'%(iou_acc), ';time:', datetime.now().replace(microsecond=0)-startTime)  
                                

                               
                        else: # isEdit=False
                            if ep==0:
                                B_loss, lr, gs, _ = sess.run([self.B_loss, self.learning_rate, 
                                                                self.global_step, self.B_optimizer], feed_dict=feed_dict)
                            else: 
                                # just show results of B with no edits
                                B_loss, lr, gs = sess.run([self.B_loss, self.learning_rate, self.global_step], feed_dict=feed_dict)

                            acc = self.accuracy(sess=sess, feed_dict=feed_dict, isEdit=isEdit)  
                            iou_acc = self.iou_accuracy(trn_data, b*self.batch_size, (b+1)*self.batch_size, 
                                                        sess=sess, feed_dict=feed_dict, isEdit=isEdit)

                            trn_acc += acc/trn_nbatch
                            B_trn_loss += B_loss/trn_nbatch
                            trn_iou += iou_acc/trn_nbatch

                            if b%50==0:
                                print('Edit:', feed_dict[self.isEdit],  ';A:',isActivateA, ';batch:', b, ';gs:', gs, ';lr: %.4f'%(lr), 
                                      ';B loss: %.3f'%(B_loss),  ';acc: %.3f'%(acc), 
                                      ';iou: %.3f'%(iou_acc),
                                      ';time:', datetime.now().replace(microsecond=0)-startTime)

                    if editRount_count>0:
                        print('\n*Training B:', ephoc%3==0, ';B Train loss: %.3f'%(B_trn_loss), ';A Train loss: %.3f'%(A_trn_loss/editRount_count),                                                                                             
                          ';Edit num: %.3f'%(edits_count/editRount_count), 
                          ';Train accuracy: %.3f'%(trn_acc),  ';IOU accuracy: %.3f'%(trn_iou), 
                          ';Time:', datetime.now().replace(microsecond=0)-startTime, '\n')
                        self.train_res.append([trn_acc, trn_iou, B_trn_loss, A_trn_loss/editRount_count])
                    else:
                        print('\n*Training B:', ephoc%3==0, ';B Train loss: %.3f'%(B_trn_loss),                                                                                          ';Train accuracy: %.3f'%(trn_acc), 
                          ';IOU accuracy: %.3f'%(trn_iou),  
                          ';Time:', datetime.now().replace(microsecond=0)-startTime, '\n')
                          
                
                self.saver.save(sess, params_dir + "/model.ckpt", global_step=ephoc)    
                if ephoc<startA or toTrainB:
                    print('Testing, ephoc:',ephoc)
                    tstTime = datetime.now().replace(microsecond=0)
                    tst_loss, tst_acc, tst_iou = 0, 0, 0
                    for b in range(tst_nbatch):
                        attn_idx, padded_queries, padded_im, padded_bbox, labels = self.build_data(tst_data,
                                                                                    b*self.batch_size, (b+1)*self.batch_size)
                        rewards_loss = np.array([[0., 0.] for _ in range(self.batch_size)])
                        feed_dict = {
                            self.queries:padded_queries,
                            self.img:padded_im,
                            self.bboxes:padded_bbox,
                            self.attn_idx:attn_idx,
                            self.labels: labels,
                            self.unk:np.array([[vocab['<unk>']]]),
                            self.reward_loss:reward_loss,
                            self.isEdit:False,
                            self.activateA:False,
                            self.Aattn_vecs:[[[]]],
                            self.dropout_in:1.,
                            self.dropout_out:1.,
                            self.dropout_img:1.,
                            self.use_wordAttn:use_wordAttn
                        }
                        B_loss = sess.run(self.B_loss, feed_dict=feed_dict)

                        acc = self.accuracy(sess=sess, feed_dict=feed_dict, isEdit=False)
                        iou_acc = self.iou_accuracy(
                            tst_data, b*self.batch_size, (b+1)*self.batch_size, sess=sess, feed_dict=feed_dict, isEdit=False)
                        tst_acc += acc/tst_nbatch
                        tst_loss += B_loss/tst_nbatch
                        tst_iou += iou_acc/tst_nbatch
                        if b%50==0:
                            print('batch:', b, ';B loss: %.3f'%(B_loss), ';acc: %.3f'%(acc), 
                                   ';iou_acc: %.3f'%(iou_acc), ';time:', datetime.now().replace(microsecond=0)-startTime)
                    print('\n*BTrain:', ephoc%3==0, ';Test loss: %.3f'%(tst_loss), ';Test accuracy %.3f'%(tst_acc), 
                          ';IOU accuracy: %.3f'%(tst_iou), ';Time:', datetime.now().replace(microsecond=0)-startTime)
                    self.test_res.append([tst_acc, tst_iou, tst_loss])
                print('='*50,'\n')
            return self.test_res, self.train_res


In [8]:
test_res, train_res = [], []

for num_hidden in [50, 100, 150, 200]:
    params_dir = params_dir_tmp+'base/hidden:'+str(num_hidden)
    tf.reset_default_graph()
    m = Model(
        batch_size=200, 
        num_hidden=num_hidden,
        embed_size=num_hidden,
        img_dims=trainset[0][1][0].shape[1], 
        bbox_dims=trainset[0][1][1].shape[1], 
        lr=.05,
        vocab=vocab, 
        decay_steps=10000, 
        decay_rate=0.99, 
        edit_reward=0.,
        rnn_editProb=0.,
        coefAlr=1,
        bnorm=False
    )


    # start comp
    print('params_dir:', params_dir)
    print('num_hidden:', m.num_hidden)
    print('learning rate:', m.lr)
    tst, trn = m.train(trainset, testset,
            ephocs_num=100,
            start_ephoc=0,
            startA=51,
            activation_ephoc=51,
            muteB=0, 
            activateAProb=0,
            max_activateAProb=0,
            editProb=0,
            edit_reward=0)
    
    test_res.append(tst)
    train_res.append(trn)
    print('\n'+'*'*100)
    print('*'*100)
    print('*'*100,'\n')

params_dir: ../data/training/models/RGAB5/All/base/hidden:50
num_hidden: 50
learning rate: 0.05
# Train set size: 59449
# Training batches: 297
# Test set size: 59507
# Testing batches: 297
Initializing variables
Train, ephoc: 0 ; Training B: True
Edit: False ;A: False ;batch: 0 ;gs: 1 ;lr: 0.0500 ;B loss: 2.757 ;acc: 0.280 ;iou: 0.370 ;time: 0:00:01
Edit: False ;A: False ;batch: 50 ;gs: 51 ;lr: 0.0500 ;B loss: 2.790 ;acc: 0.215 ;iou: 0.325 ;time: 0:00:10
Edit: False ;A: False ;batch: 100 ;gs: 101 ;lr: 0.0500 ;B loss: 2.709 ;acc: 0.195 ;iou: 0.320 ;time: 0:00:19
Edit: False ;A: False ;batch: 150 ;gs: 151 ;lr: 0.0500 ;B loss: 2.690 ;acc: 0.215 ;iou: 0.295 ;time: 0:00:27
Edit: False ;A: False ;batch: 200 ;gs: 201 ;lr: 0.0500 ;B loss: 2.740 ;acc: 0.215 ;iou: 0.285 ;time: 0:00:37
Edit: False ;A: False ;batch: 250 ;gs: 251 ;lr: 0.0500 ;B loss: 2.717 ;acc: 0.225 ;iou: 0.330 ;time: 0:00:46

*Training B: True ;B Train loss: 2.716 ;Train accuracy: 0.229 ;IOU accuracy: 0.322 ;Time: 0:00:56 

Tes

In [14]:
bi_test_res, bi_train_res = [], []

for num_hidden in [50, 100, 150, 200]:
    params_dir = params_dir_tmp+'base/Bi_hidden:'+str(num_hidden)
    tf.reset_default_graph()
    m = Model(
        batch_size=200, 
        num_hidden=num_hidden,
        embed_size=num_hidden,
        img_dims=trainset[0][1][0].shape[1], 
        bbox_dims=trainset[0][1][1].shape[1], 
        lr=.05,
        vocab=vocab, 
        decay_steps=10000, 
        decay_rate=0.99, 
        edit_reward=0.,
        rnn_editProb=0.,
        coefAlr=1,
        useBidirectionalRnn=True,
        bnorm=False
    )


    # start comp
    print('params_dir:', params_dir)
    print('num_hidden:', m.num_hidden)
    print('learning rate:', m.lr)
    tst, trn = m.train(trainset, testset,
            ephocs_num=100,
            start_ephoc=0,
            startA=51,
            activation_ephoc=51,
            muteB=0, 
            activateAProb=0,
            max_activateAProb=0,
            editProb=0,
            edit_reward=0)
    
    bi_test_res.append(tst)
    bi_train_res.append(trn)
    print('\n'+'*'*100)
    print('*'*100)
    print('*'*100,'\n')

ValueError: Shape must be rank 3 but is rank 4 for 'Slice' (op: 'Slice') with input shapes: [2,200,?,100], [3], [3].

In [12]:
bnorm_test_res, bnorm_train_res = [], []

for num_hidden in [50, 100, 150, 200]:
    params_dir = params_dir_tmp+'bnorm_base/hidden:'+str(num_hidden)
    tf.reset_default_graph()
    m = Model(
        batch_size=200, 
        num_hidden=num_hidden,
        embed_size=num_hidden,
        img_dims=trainset[0][1][0].shape[1], 
        bbox_dims=trainset[0][1][1].shape[1], 
        lr=.05,
        vocab=vocab, 
        decay_steps=10000, 
        decay_rate=0.99, 
        edit_reward=0.,
        rnn_editProb=0.,
        coefAlr=1,
        bnorm=True)


    # start comp
    print('params_dir:', params_dir)
    print('num_hidden:', m.num_hidden)
    print('learning rate:', m.lr)
    tst, trn = m.train(trainset, testset,
            ephocs_num=50,
            start_ephoc=0,
            startA=51,
            activation_ephoc=51,
            muteB=0, 
            activateAProb=0,
            max_activateAProb=0,
            editProb=0,
            edit_reward=0)
    
    bnorm_test_res.append(tst)
    bnorm_train_res.append(trn)
    print('\n'+'*'*100)
    print('*'*100)
    print('*'*100,'\n')

params_dir: ../data/training/models/RGAB5/All/bnorm_base/hidden:50
num_hidden: 50
learning rate: 0.05
# Train set size: 59449
# Training batches: 297
# Test set size: 59507
# Testing batches: 297
Initializing variables
Train, ephoc: 0 ; Training B: True
Edit: False ;A: False ;batch: 0 ;gs: 1 ;lr: 0.0500 ;B loss: 2.759 ;acc: 0.325 ;iou: 0.395 ;time: 0:00:00
Edit: False ;A: False ;batch: 50 ;gs: 51 ;lr: 0.0500 ;B loss: 2.695 ;acc: 0.240 ;iou: 0.375 ;time: 0:00:08
Edit: False ;A: False ;batch: 100 ;gs: 101 ;lr: 0.0500 ;B loss: 2.664 ;acc: 0.285 ;iou: 0.405 ;time: 0:00:16
Edit: False ;A: False ;batch: 150 ;gs: 151 ;lr: 0.0500 ;B loss: 2.671 ;acc: 0.320 ;iou: 0.435 ;time: 0:00:25
Edit: False ;A: False ;batch: 200 ;gs: 201 ;lr: 0.0500 ;B loss: 2.539 ;acc: 0.380 ;iou: 0.455 ;time: 0:00:34
Edit: False ;A: False ;batch: 250 ;gs: 251 ;lr: 0.0500 ;B loss: 2.474 ;acc: 0.400 ;iou: 0.470 ;time: 0:00:43

*Training B: True ;B Train loss: 2.625 ;Train accuracy: 0.319 ;IOU accuracy: 0.419 ;Time: 0:00:51

In [17]:
#dropout_in
dout_bnorm_test_res, dout_bnorm_train_res = [], []
drpos_ratio = [[0.5, 1.], [1., 0.5], [.5,.5]]
for num_hidden in [50, 100, 150, 200]:
    for drops in drpos_ratio:
        dropout_in, dropout_out = drops
        params_dir = params_dir_tmp+'bnorm_base/drop_'+str(dropout_in)+','+str(dropout_out)+'_hidden:'+str(num_hidden)
        tf.reset_default_graph()
        m = Model(
            batch_size=200, 
            num_hidden=num_hidden,
            embed_size=num_hidden,
            img_dims=trainset[0][1][0].shape[1], 
            bbox_dims=trainset[0][1][1].shape[1], 
            lr=.05,
            vocab=vocab, 
            decay_steps=10000, 
            decay_rate=0.99, 
            edit_reward=0.,
            rnn_editProb=0.,
            coefAlr=1,
            bnorm=True)


        # start comp
        print('dropout_in', dropout_in)
        print('dropout_out', dropout_out)
        print('params_dir:', params_dir)
        print('num_hidden:', m.num_hidden)
        print('learning rate:', m.lr)
        tst, trn = m.train(trainset, testset,
                ephocs_num=50,
                start_ephoc=0,
                startA=51,
                activation_ephoc=51,
                muteB=0, 
                activateAProb=0,
                max_activateAProb=0,
                editProb=0,
                edit_reward=0,
                dropout_in=dropout_in,
                dropout_out=dropout_out)

        dout_bnorm_test_res.append(tst)
        dout_bnorm_train_res.append(trn)
        print('\n'+'*'*100)
        print('*'*100)
        print('*'*100,'\n')

dropout_in 0.5
dropout_out 1.0
params_dir: ../data/training/models/RGAB5/All/bnorm_base/drop_0.5,1.0_hidden:50
num_hidden: 50
learning rate: 0.05
# Train set size: 59449
# Training batches: 297
# Test set size: 59507
# Testing batches: 297
Initializing variables
Train, ephoc: 0 ; Training B: True
Edit: False ;A: False ;batch: 0 ;gs: 1 ;lr: 0.0500 ;B loss: 2.942 ;acc: 0.315 ;iou: 0.380 ;time: 0:00:00
Edit: False ;A: False ;batch: 50 ;gs: 51 ;lr: 0.0500 ;B loss: 2.743 ;acc: 0.235 ;iou: 0.345 ;time: 0:00:09
Edit: False ;A: False ;batch: 100 ;gs: 101 ;lr: 0.0500 ;B loss: 2.614 ;acc: 0.300 ;iou: 0.445 ;time: 0:00:18
Edit: False ;A: False ;batch: 150 ;gs: 151 ;lr: 0.0500 ;B loss: 2.658 ;acc: 0.320 ;iou: 0.435 ;time: 0:00:28
Edit: False ;A: False ;batch: 200 ;gs: 201 ;lr: 0.0500 ;B loss: 2.693 ;acc: 0.310 ;iou: 0.425 ;time: 0:00:37
Edit: False ;A: False ;batch: 250 ;gs: 251 ;lr: 0.0500 ;B loss: 2.491 ;acc: 0.310 ;iou: 0.475 ;time: 0:00:47

*Training B: True ;B Train loss: 2.639 ;Train accurac

In [8]:
# dropout_in +  dropout_in + drop_img
dimg_bnorm_test_res, dimg_bnorm_train_res = [], []
num_hidden=200
drpos_ratio = [[0.5, 1.], [1., 0.5], [.5,.5]]
for drops in drpos_ratio:
    dropout_in, dropout_out = drops
    params_dir = params_dir_tmp+'bnorm_base/img_drop_'+str(dropout_in)+','+str(dropout_out)+'_hidden:'+str(num_hidden)
    tf.reset_default_graph()
    m = Model(
        batch_size=200, 
        num_hidden=num_hidden,
        embed_size=num_hidden,
        img_dims=trainset[0][1][0].shape[1], 
        bbox_dims=trainset[0][1][1].shape[1], 
        lr=.05,
        vocab=vocab, 
        decay_steps=10000, 
        decay_rate=0.99, 
        edit_reward=0.,
        rnn_editProb=0.,
        coefAlr=1,
        bnorm=True)


    # start comp
    print('dropout_in', dropout_in)
    print('dropout_out', dropout_out)
    print('params_dir:', params_dir)
    print('num_hidden:', m.num_hidden)
    print('learning rate:', m.lr)
    tst, trn = m.train(trainset, testset,
            ephocs_num=70,
            start_ephoc=0,
            startA=71,
            activation_ephoc=71,
            muteB=0, 
            activateAProb=0,
            max_activateAProb=0,
            editProb=0,
            edit_reward=0,
            dropout_img=0.5,
            dropout_in=dropout_in,
            dropout_out=dropout_out)

    dimg_bnorm_test_res.append(tst)
    dimg_bnorm_train_res.append(trn)
    print('\n'+'*'*100)
    print('*'*100)
    print('*'*100,'\n')                                                             

dropout_in 0.5
dropout_out 1.0
params_dir: ../data/training/models/RGAB5/All/bnorm_base/img_drop_0.5,1.0_hidden:200
num_hidden: 200
learning rate: 0.05
# Train set size: 59449
# Training batches: 297
# Test set size: 59507
# Testing batches: 297
Initializing variables
Train, ephoc: 0 ; Training B: True
Edit: False ;A: False ;batch: 0 ;gs: 1 ;lr: 0.0500 ;B loss: 2.971 ;acc: 0.315 ;iou: 0.400 ;time: 0:00:01
Edit: False ;A: False ;batch: 50 ;gs: 51 ;lr: 0.0500 ;B loss: 2.728 ;acc: 0.230 ;iou: 0.330 ;time: 0:00:11
Edit: False ;A: False ;batch: 100 ;gs: 101 ;lr: 0.0500 ;B loss: 2.845 ;acc: 0.250 ;iou: 0.330 ;time: 0:00:19
Edit: False ;A: False ;batch: 150 ;gs: 151 ;lr: 0.0500 ;B loss: 2.726 ;acc: 0.280 ;iou: 0.400 ;time: 0:00:29
Edit: False ;A: False ;batch: 200 ;gs: 201 ;lr: 0.0500 ;B loss: 2.560 ;acc: 0.360 ;iou: 0.440 ;time: 0:00:38
Edit: False ;A: False ;batch: 250 ;gs: 251 ;lr: 0.0500 ;B loss: 2.666 ;acc: 0.280 ;iou: 0.370 ;time: 0:00:47

*Training B: True ;B Train loss: 2.690 ;Train a

In [11]:
#dropout_in
noise_bnorm_test_res, noise_bnorm_train_res = [], []
num_hidden = 200
dropout_in=0.5
dropout_out=1.
dropout_img = 0.5
params_dir = params_dir_tmp+'bnorm_base/noise_'+str(dropout_in)+','+str(dropout_out)+','+str(dropout_img)+'_hidden:'+str(num_hidden)

tf.reset_default_graph()
m = Model(
    batch_size=200, 
    num_hidden=num_hidden,
    embed_size=num_hidden,
    img_dims=trainset[0][1][0].shape[1], 
    bbox_dims=trainset[0][1][1].shape[1], 
    lr=.05,
    vocab=vocab, 
    decay_steps=10000, 
    decay_rate=0.99, 
    edit_reward=0.,
    rnn_editProb=0.,
    coefAlr=1,
    bnorm=True)


# start comp
print('params_dir:', params_dir)
print('num_hidden:', m.num_hidden)
print('learning rate:', m.lr)
tst, trn = m.train(trainset, testset,
        ephocs_num=70,
        start_ephoc=0,
        startA=101,
        activation_ephoc=101,
        muteB=0, 
        activateAProb=0,
        max_activateAProb=0,
        editProb=0,
        edit_reward=0,
        dropout_img=dropout_img,
        dropout_in=dropout_in,
        dropout_out=dropout_out,
        addNoise=True)

noise_bnorm_test_res.append(tst)
noise_bnorm_train_res.append(trn)
print('\n'+'*'*100)
print('*'*100)
print('*'*100,'\n')

params_dir: ../data/training/models/RGAB5/All/bnorm_base/noise_0.5,1.0,0.5_hidden:200
num_hidden: 200
learning rate: 0.05
# Train set size: 59449
# Training batches: 297
# Test set size: 59507
# Testing batches: 297
Initializing variables
Train, ephoc: 0 ; Training B: True
Edit: False ;A: False ;batch: 0 ;gs: 1 ;lr: 0.0500 ;B loss: 3.015 ;acc: 0.345 ;iou: 0.440 ;time: 0:00:01
Edit: False ;A: False ;batch: 50 ;gs: 51 ;lr: 0.0500 ;B loss: 2.766 ;acc: 0.275 ;iou: 0.345 ;time: 0:01:05
Edit: False ;A: False ;batch: 100 ;gs: 101 ;lr: 0.0500 ;B loss: 2.724 ;acc: 0.270 ;iou: 0.365 ;time: 0:02:09
Edit: False ;A: False ;batch: 150 ;gs: 151 ;lr: 0.0500 ;B loss: 2.709 ;acc: 0.235 ;iou: 0.355 ;time: 0:03:13
Edit: False ;A: False ;batch: 200 ;gs: 201 ;lr: 0.0500 ;B loss: 2.721 ;acc: 0.315 ;iou: 0.450 ;time: 0:04:20
Edit: False ;A: False ;batch: 250 ;gs: 251 ;lr: 0.0500 ;B loss: 2.694 ;acc: 0.280 ;iou: 0.395 ;time: 0:05:27

*Training B: True ;B Train loss: 2.708 ;Train accuracy: 0.277 ;IOU accuracy: 

KeyboardInterrupt: 

In [ ]:
#dropout_in
noise_bnorm_test_res, noise_bnorm_train_res = [], []
num_hidden = 200
dropout_in=0.5
dropout_out=1.
dropout_img = 0.5
params_dir = params_dir_tmp+'bnorm_base/test_noise_'+str(dropout_in)+','+str(dropout_out)+','+str(dropout_img)+'_hidden:'+str(num_hidden)

tf.reset_default_graph()
m = Model(
    batch_size=200, 
    num_hidden=num_hidden,
    embed_size=num_hidden,
    img_dims=trainset[0][1][0].shape[1], 
    bbox_dims=trainset[0][1][1].shape[1], 
    lr=.05,
    vocab=vocab, 
    decay_steps=10000, 
    decay_rate=0.99, 
    edit_reward=0.,
    rnn_editProb=0.,
    coefAlr=1,
    bnorm=True)


# start comp
print('params_dir:', params_dir)
print('num_hidden:', m.num_hidden)
print('learning rate:', m.lr)
tst, trn = m.train(trainset, testset,
        ephocs_num=70,
        start_ephoc=0,
        startA=101,
        activation_ephoc=101,
        muteB=0, 
        activateAProb=0,
        max_activateAProb=0,
        editProb=0,
        edit_reward=0,
        dropout_img=dropout_img,
        dropout_in=dropout_in,
        dropout_out=dropout_out,
        addNoise=True)

noise_bnorm_test_res.append(tst)
noise_bnorm_train_res.append(trn)
print('\n'+'*'*100)
print('*'*100)
print('*'*100,'\n')

params_dir: ../data/training/models/RGAB5/All/bnorm_base/test_noise_0.5,1.0,0.5_hidden:200
num_hidden: 200
learning rate: 0.05
# Train set size: 59449
# Training batches: 297
# Test set size: 59507
# Testing batches: 297
Initializing variables
Train, ephoc: 0 ; Training B: True
Edit: False ;A: False ;batch: 0 ;gs: 1 ;lr: 0.0500 ;B loss: 2.979 ;acc: 0.265 ;iou: 0.420 ;time: 0:00:02
Edit: False ;A: False ;batch: 50 ;gs: 51 ;lr: 0.0500 ;B loss: 2.686 ;acc: 0.310 ;iou: 0.430 ;time: 0:01:04
Edit: False ;A: False ;batch: 100 ;gs: 101 ;lr: 0.0500 ;B loss: 2.684 ;acc: 0.280 ;iou: 0.380 ;time: 0:02:07
Edit: False ;A: False ;batch: 150 ;gs: 151 ;lr: 0.0500 ;B loss: 2.673 ;acc: 0.305 ;iou: 0.410 ;time: 0:03:09
Edit: False ;A: False ;batch: 200 ;gs: 201 ;lr: 0.0500 ;B loss: 2.777 ;acc: 0.280 ;iou: 0.390 ;time: 0:04:15
Edit: False ;A: False ;batch: 250 ;gs: 251 ;lr: 0.0500 ;B loss: 2.605 ;acc: 0.340 ;iou: 0.435 ;time: 0:05:15

*Training B: True ;B Train loss: 2.708 ;Train accuracy: 0.284 ;IOU accur